In [1]:
import pytrec_eval
import pandas as pd
import os
import numpy as np
import ir_datasets
import json

In [2]:
def load_runs_metadata_table():
    table = []
    for run in os.listdir(f"../data/run"):
        parts = run.split("-")
        fields = {
            "dataset": "-".join(parts[1:-4]),
            "subcollection": parts[-4],
            "queries": parts[-3],
            "method": parts[-2],
            "implementation": parts[-1],
            "filename": run,
        }
        table.append(fields)
    runs = pd.DataFrame(table)
    runs = runs[
        ~((runs["subcollection"] == "WT") & (runs["queries"] != "queries"))
    ]  # longeval WT test only
    runs = runs[
        runs["method"].isin(
            [
                "bm25",
                "bm25+colbert",
                "bm25+monot5",
                "rrf(xsqram__bm25_bo1__pl2)",
                "bm25_d2q10",
            ]
        )
    ]
    return runs

# Trec-Covid

In [5]:
runs_table = load_runs_metadata_table()

In [146]:
for _, row in runs_table[runs_table["dataset"] =="trec-covid"].iterrows():
    run = pd.read_csv("../data/run/"+row.filename, sep=" ", header=None, names=["qid", "Q0", "docid", "rank", "score", "method"])
    run = run[run["qid"]<=30]
    run.to_csv("../data/run-core_queries/"+row.filename, sep=" ", header=None, index=False)

# LongEval
### Merge train and test runs for $t_0$ WT runs

In [4]:
table = []
for run in os.listdir("../data/run"):
    parts = run.split("-")
    fields = {"dataset":  "-".join(parts[1:-4]),
    "subcollection":  parts[-4],
    "queries": parts[-3],
    "method": parts[-2],
    "implementation": parts[-1],
    "filename": run}
    table.append(fields)

In [5]:
df = pd.DataFrame(table)

In [36]:
for method in df[df["dataset"]=="longeval"][df["subcollection"]=="WT"]["method"].unique():
    runs = df[df["dataset"]=="longeval"][df["subcollection"]=="WT"][df["method"]==method]["filename"].to_list()
    implementation = runs[0].split("-")[-1]
    full_run = pd.concat([pd.read_csv(base_path+runs[0], sep=" ", header=None, names=["qid", "Q0", "docid", "rank", "score", "method"]), pd.read_csv(base_path+runs[1], sep=" ", header=None, names=["qid", "Q0", "docid", "rank", "score", "method"])])
    full_run.to_csv(f"../data/run/run-longeval-WT-queries-{method}-{implementation}", sep=" ", header=None, index=False)

/tmp/ipykernel_28003/3578907483.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for method in df[df["dataset"]=="longeval"][df["subcollection"]=="WT"]["method"].unique():
/tmp/ipykernel_28003/3578907483.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  runs = df[df["dataset"]=="longeval"][df["subcollection"]=="WT"][df["method"]==method]["filename"].to_list()
/tmp/ipykernel_28003/3578907483.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  runs = df[df["dataset"]=="longeval"][df["subcollection"]=="WT"][df["method"]==method]["filename"].to_list()
/tmp/ipykernel_28003/3578907483.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  runs = df[df["dataset"]=="longeval"][df["subcollection"]=="WT"][df["method"]==method]["filename"].to_list()
/tmp/ipykernel_28003/3578907483.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  runs 

### Filter runs

In [114]:
def filter_run(row, core_docs=False):
    run = pd.read_csv("../data/run/"+row.filename, sep=" ", header=None, names=["qid", "Q0", "docid", "rank", "score", "method"])
    
    # prepare patches
    longeval_core_docs = pd.read_csv("../evaluation/core_docs.tsv", sep="\t", index_col=0)
    patch_doc = longeval_core_docs[f"docno_{row.subcollection}"].reset_index()[[f"docno_{row.subcollection}", "index"]].set_index(f"docno_{row.subcollection}")["index"].astype(str).to_dict()

    longeval_core_queries = pd.read_csv("../evaluation/core_queries.tsv", sep="\t", index_col=0)
    patch_queries = longeval_core_queries[f"qid_{row.subcollection}"].reset_index()[[f"qid_{row.subcollection}", "index"]].set_index(f"qid_{row.subcollection}")["index"].astype(str).to_dict()

    if core_docs:
        run["docid"] = run["docid"].apply(lambda x: patch_doc.get(x, None))
    else:
        run["docid"] = run["docid"].apply(lambda x: patch_doc.get(x, x))
    run["qid"] = run["qid"].apply(lambda x: patch_queries.get(x, None))

    return run.dropna()

In [117]:
runs_table = load_runs_metadata_table()
for _, row in runs_table[runs_table["dataset"] =="longeval"].iterrows():
    run = filter_run(row)
    run.to_csv("../data/run-core_queries/"+row.filename, sep=" ", header=None, index=False)

# TripClick

In [173]:
for _, row in runs_table[runs_table["dataset"] =="tripclick-test-head"].iterrows():
    run = pd.read_csv("../data/run/"+row.filename, sep=" ", header=None, names=["qid", "Q0", "docid", "rank", "score", "method"])
    run.to_csv("../data/run-core_queries/"+row.filename, sep=" ", header=None, index=False)